In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/My Drive/Google Colaboratory/the-catcher-in-the-rye/

/content/gdrive/My Drive/Google Colaboratory/the-catcher-in-the-rye


In [4]:
%ls -al

total 34
-rw------- 1 root root  140 Aug 14 06:33 apostrophe_dict.txt
drwx------ 2 root root 4096 Aug 23 05:19 ebook/
-rw------- 1 root root 1062 Aug  3 13:08 LICENSE
drwx------ 2 root root 4096 Aug 23 05:19 nltk_data/
drwx------ 2 root root 4096 Aug 23 05:19 notebooks/
drwx------ 2 root root 4096 Aug 23 05:19 output/
-rw------- 1 root root 2473 Aug 14 07:56 preprocess.py
-rw------- 1 root root 2021 Aug  6 13:47 question1.py
-rw------- 1 root root 4122 Aug 22 14:08 question2.py
-rw------- 1 root root 2801 Aug 22 14:24 README.md
-rw------- 1 root root 1879 Aug 23 05:27 requirements.txt
-rw------- 1 root root 1303 Aug  6 13:39 utils.py


In [5]:
!pip install --upgrade pip

     |████████████████████████████████| 1.5MB 4.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [6]:
!pip install ebooklib

     |████████████████████████████████| 111 kB 4.6 MB/s 
  Created wheel for ebooklib: filename=EbookLib-0.17.1-py3-none-any.whl size=38164 sha256=c92d4ea62e9ab61c7479fcd1190b8f3c9ccab7bbeafc9c8a63b2cb5408a7ce0d
  Stored in directory: /root/.cache/pip/wheels/b4/83/65/67f90f3f9cdd3eb5330e495fbbd488bc3c0854f76eed1b704c
Successfully built ebooklib


In [7]:
from collections import defaultdict
import re

from bs4 import BeautifulSoup
from ebooklib import epub
import ebooklib


def transform_epub_to_html(epub_path):
    book = epub.read_epub(epub_path)
    chapters = defaultdict(str)
    for item in book.get_items():
        name = item.get_name()
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            chapters[name] = item.get_content()
    return chapters


def transform_html_to_text(html):
    blacklist = [
        '[document]', 'noscript', 'header', 'html',
        'meta', 'head', 'input', 'script'
    ]
    soup = BeautifulSoup(html, 'html.parser')
    output = ''
    for text in soup.find_all(text=True):
        if text.parent.name in blacklist:
            continue
        output += (' ' + text)
    return output


def load_epub(epub_path):
    chapters = transform_epub_to_html(epub_path)
    book = defaultdict(str)
    for chapter, html in chapters.items():
        chapter_found = re.search(
            r'The_Catcher_in_the_Rye_split_([\d]{3}).html', chapter
        )
        if chapter_found:
            chapter = int(chapter_found.group(1))
            text = transform_html_to_text(html)
            text = re.sub(r'[\s][\s]+', ' ', text)
            text = text.strip()
            book[chapter] = text
    return book

In [8]:
from collections import Counter
from collections import defaultdict
import re

from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pandas as pd


def _nltk_settings(data_path):
    nltk.data.path.append(data_path)


def _save_apostrophe_dataframe(book):
    text = [t.lower() for t in book.values()]
    text = ' '.join(text)
    pattern = r'([\w]*\'[\w]*)'
    apostrophes = re.findall(pattern, text)
    counter = Counter(apostrophes)
    apostrophe_df = pd.DataFrame(
        counter.most_common(),
        columns=['apostrophe', 'frequency']
    )

    return apostrophe_df


def _get_apostrophe_dict(filename):
    with open(filename, 'r') as f:
        apostrophe_dict = defaultdict(str)
        for line in f.readlines():
            src, dst = line.split('\t')
            apostrophe_dict[src] = dst
    return apostrophe_dict


def process_apostrophe(text, filename='./apostrophe_dict.txt'):
    apostrophe_dict = _get_apostrophe_dict(filename)
    for src, dst in apostrophe_dict.items():
        text = text.replace(src, dst)
    return text


def _get_wordnet_pos(pos):
    if pos.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif pos.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif pos.startswith('R'):
        return nltk.corpus.wordnet.ADV

    return nltk.corpus.wordnet.NOUN


def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = list()
    for word, pos in pos_tag(wordpunct_tokenize(text)):
        wordnet_pos = _get_wordnet_pos(pos)
        word = lemmatizer.lemmatize(word, pos=wordnet_pos)
        if re.match(r'[\w]+', word):
            words.append(word)
    return ' '.join(words)


def process_text_data(book):
    _nltk_settings(data_path='./nltk_data/')
    _save_apostrophe_dataframe(book)

    book_processed = defaultdict(list)

    for chapter in book.keys():
        text = book[chapter]
        text = text.lower()

        text = process_apostrophe(text)
        tokenizer = PunktSentenceTokenizer()
        for sentence in tokenizer.tokenize(text):
            sentence = lemmatize_text(sentence)
            if len(sentence):
                book_processed[chapter].append(sentence)
    return book_processed


In [9]:
epub_path = \
    './ebook/J. D. Salinger - The Catcher in the Rye '\
    '(1951, Penguin Books Ltd).epub'
book = load_epub(epub_path)
book = process_text_data(book)

In [10]:
book[0] = [' '.join(book[0])]

In [40]:
from nltk.corpus import stopwords

corpus = list()
stopword = stopwords.words('english')
for _, sentences in book.items():
    for sentence in sentences:
        stopword_removal = list()
        for word in sentence.split():
            if word in stopword:
                continue
            stopword_removal.append(word)
        if len(stopword_removal) > 1:
            corpus.append(' '.join(stopword_removal))
len(corpus)

5897

In [41]:
corpus[101]

'even know run guess felt like'

In [42]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

word2idx = tokenizer.word_index
encoded = tokenizer.texts_to_sequences(corpus)

encoded[101]

[16, 6, 245, 219, 83, 4]

In [43]:
vocab_size = len(word2idx) + 1
vocab_size

3085

In [45]:
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Activation, Dot, Embedding, Reshape
from tensorflow.keras.preprocessing.sequence import skipgrams
from tqdm import tqdm
import numpy as np

class Word2Vec:
    def __init__(self, vocab_size, embedding_dim=30):
        self.model = self.build_model(vocab_size, embedding_dim)
        self.model.compile(loss='binary_crossentropy', optimizer='adam')

    def build_model(self, vocab_size, embedding_dim):
        center_model = keras.models.Sequential()
        context_model = keras.models.Sequential()

        center_model.add(
            Embedding(vocab_size, embedding_dim, name='embedding_center')
        )
        context_model.add(
            Embedding(vocab_size, embedding_dim, name='embedding_context')
        )

        center_word = keras.Input(shape=(1,))
        context_word = keras.Input(shape=(1,))

        center_embedding = center_model(center_word)
        context_embedding = context_model(context_word)

        word2vec = keras.models.Sequential()
        word2vec.add(Dot(axes=2, name='dot'))
        word2vec.add(Reshape((1,), name='reshape'))
        word2vec.add(Activation('sigmoid', name='sigmoid'))

        output = word2vec([center_embedding, context_embedding])

        return Model([center_word, context_word], output)

    def train(self, encoded, epochs=10):
        for epoch in range(epochs):
            loss = 0.0
            for sentence in tqdm(encoded):
                sg = skipgrams(
                    sentence, vocabulary_size=vocab_size, window_size=5
                )

                center_elem = np.array(list(zip(*sg[0]))[0])
                context_elem = np.array(list(zip(*sg[0]))[1])
                labels = np.array(sg[1])
                X = [center_elem, context_elem]
                Y = labels

                loss += self.model.train_on_batch(X, Y)

            print(f'Epoch : {epoch}, Loss: {loss:.4f}')

In [46]:
model = Word2Vec(vocab_size=vocab_size)
model.train(encoded, epochs=10)

Consider rewriting this model with the Functional API.


  0%|          | 0/5897 [00:00<?, ?it/s]

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.


  0%|          | 1/5897 [00:00<41:08,  2.39it/s]

Consider rewriting this model with the Functional API.


...

Epoch : 7, Loss: 2177.2093


  0%|          | 13/5897 [00:00<00:45, 128.59it/s]

Epoch : 8, Loss: 2102.2799


100%|██████████| 5897/5897 [00:42<00:00, 137.22it/s]

Epoch : 9, Loss: 2049.0873


In [49]:
model.model.get_weights()[0].shape

(3085, 30)

In [53]:
with open('output/weights/word_embedding.txt', 'w') as f:
    vectors = model.model.get_weights()[0]
    f.write(f'{vectors.shape[0]-1} {vectors.shape[1]}\n')
    for word, i in tokenizer.word_index.items():
        word_vec = ' '.join(map(str, list(vectors[i, :])))
        f.write(f'{word} {word_vec}\n')

In [47]:
import gensim

In [55]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('output/weights/word_embedding.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [56]:
w2v.most_similar(positive=['school'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('insulted', 0.8151702880859375),
 ('sister', 0.8074385523796082),
 ('vocabulary', 0.7733476161956787),
 ('whooton', 0.7665121555328369),
 ('father', 0.762919545173645),
 ('louis', 0.7629117965698242),
 ('prep', 0.7627164721488953),
 ('grade', 0.7603752613067627),
 ('shaney', 0.7599301338195801),
 ('full', 0.7528494000434875)]

In [70]:
w2v.most_similar(positive=['holden'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('shirley', 0.7938888072967529),
 ('phoebe', 0.7632480263710022),
 ('accordingly', 0.7458309531211853),
 ('record', 0.7239189743995667),
 ('mob', 0.7231490612030029),
 ('bean', 0.7226176261901855),
 ('quarter', 0.7223073840141296),
 ('humility', 0.7200443744659424),
 ('ceety', 0.7182319164276123),
 ('beeg', 0.713215708732605)]

In [71]:
w2v.most_similar(positive=['ackley'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('besides', 0.7177923917770386),
 ('thinking', 0.717034101486206),
 ('stradlater', 0.6901248693466187),
 ('reason', 0.6392755508422852),
 ('habit', 0.6389061212539673),
 ('slap', 0.6345095038414001),
 ('wanna', 0.6211247444152832),
 ('bastard', 0.6187803745269775),
 ('marry', 0.6184471249580383),
 ('unless', 0.6175366640090942)]

In [72]:
w2v.most_similar(positive=['stradlater'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('gatsby', 0.7059965133666992),
 ('ackley', 0.6901246309280396),
 ('whistle', 0.6735602021217346),
 ('shower', 0.669865608215332),
 ('piercing', 0.6503495573997498),
 ('slob', 0.6362868547439575),
 ('reason', 0.6252253651618958),
 ('classical', 0.6125445365905762),
 ('besides', 0.601589024066925),
 ('pinscher', 0.5956303477287292)]

In [74]:
w2v.most_similar(positive=['jane'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('jean', 0.8745556473731995),
 ('friend', 0.6977934837341309),
 ('lining', 0.6620803475379944),
 ('extremely', 0.6562008857727051),
 ('late', 0.6530422568321228),
 ('entertain', 0.6495700478553772),
 ('4th', 0.641986608505249),
 ('either', 0.633868932723999),
 ('sure', 0.6320369243621826),
 ('license', 0.6304012537002563)]

In [77]:
w2v.most_similar(positive=['caulfield'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('mother', 0.8288822770118713),
 ('jesus', 0.7397575378417969),
 ('oh', 0.7336921095848083),
 ('schmidt', 0.7319296598434448),
 ('4b', 0.7301366329193115),
 ('riviera', 0.7147058844566345),
 ('v', 0.7011089324951172),
 ('weatherfield', 0.696500837802887),
 ('hazel', 0.6916140913963318),
 ('slightly', 0.6782609820365906)]

In [80]:
w2v.most_similar(positive=['rye'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('louis', 0.8116165399551392),
 ('create', 0.7826299071311951),
 ('catch', 0.7706725001335144),
 ('simple', 0.7674775719642639),
 ('stabile', 0.7590072751045227),
 ('french', 0.7546792030334473),
 ('squeal', 0.7456923723220825),
 ('sister', 0.7428297996520996),
 ('insulted', 0.7412819862365723),
 ('whooton', 0.7408576607704163)]

In [81]:
w2v.most_similar(positive=['sister'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('spell', 0.8873235583305359),
 ('insulted', 0.8823113441467285),
 ('aspirin', 0.8813509345054626),
 ('wealthy', 0.8798375725746155),
 ('gentleman', 0.8622280359268188),
 ('whore', 0.8585451245307922),
 ('grade', 0.8486475944519043),
 ('tea', 0.8471783399581909),
 ('yank', 0.8442176580429077),
 ('footstep', 0.8434706926345825)]